# Building a VAE for shits n Giggles

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, backend

###  setting constants

In [2]:
latent_dim = 2

# Step 1 - building Sampling Function

Becuase VAE need to sample from a distribution, we need to build a function that will sample from that distribution at the same dim as the input  
https://keras.io/examples/generative/vae/

In [8]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


    
def sampling(mu, sigma):
    """Direct call to the sampling function instead of a class layers.Layer (like above)"""
    batch = tf.shape(mu)[0]
    dim = tf.shape(mu)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return mu + tf.exp(0.5 * sigma) * epsilon

# Step 1 - building a model
#### Encoder
We're going to use  a sequentail model to get this done, but we are going to split it into the Encoder/Decoder aspect

I want you to notice the `layers.Dense(latent_dim + latent_dim)` part. Remeber, we are building a guassian distribution distribution to pull one, so we have two variables **per gaussian distribution** to optimize for - mean and std. So latent_dim x (mean, std) = latent_dim * mean + latent_dim * std 

In [7]:
vae_encoder = tf.keras.Sequential([
    layers.InputLayer(input_shape=(28, 28, 1)),
    layers.Conv2D(filters=8, 
                  kernel_size=(3,3),
                  #strides=(2, 2), 
                  activation='relu'),
    layers.Conv2D(filters=32, 
                  kernel_size=(3,3),
                  #strides=(2, 2), 
                  activation='relu'),
     layers.Conv2D(filters=64,
                   kernel_size=(3,3), 
                   #strides=(2, 2), 
                   activation='relu'),
     tf.keras.layers.Flatten(),
     layers.Dense(16, activation = 'tanh'),
     layers.Dense(latent_dim + latent_dim) #first latent_dim is for mu, second is for std
     
        ]
    )


In [4]:
?layers.Conv2D


Init signature:
layers.Conv2D(
    filters,
    kernel_size,
    strides=(1, 1),
    padding='valid',
    data_format=None,
    dilation_rate=(1, 1),
    activation=None,
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs,
)
Docstring:     
2D convolution layer (e.g. spatial convolution over images).

This layer creates a convolution kernel that is convolved
with the layer input to produce a tensor of
outputs. If `use_bias` is True,
a bias vector is created and added to the outputs. Finally, if
`activation` is not `None`, it is applied to the outputs as well.

When using this layer as the first layer in a model,
provide the keyword argument `input_shape`
(tuple of integers, does not include the sample axis),
e.g. `input_shape=(128, 128, 3)` for 128x128 RGB pictures
in `data_format="channels_last"